In [49]:
import numpy as np
import re 
from bs4 import BeautifulSoup
from datetime import datetime
#from sklearn.linear_model import LinearRegression

def get_numbers_only(input_string, mode): 
    '''Used to filter megawatt hours and date and TIME from the html code'''
    if mode == "date":
        mode_code = r'\b\d{4}-\d{2}-\d{2}\b'
    elif mode == "mwh":
        mode_code = r'[\d,]+'
    else: 
        return "incorrect code, please check"

    match = re.search(mode_code, input_string)
    #Check if a match was found
    if match:
        #Remove commas from the matched string and convert it to an integer
        number = match.group().replace(',', '')
        return number
    else:
        print("No number found in the input string.")

In [9]:
file_object = open("battery-report.html", "r")
html_data = file_object.read()
#print(data)
file_object.close()
data = BeautifulSoup(html_data, 'html.parser')

In [20]:
target_text = "Battery capacity history"
h2_element = data.find("h2", string=lambda text: text and text.strip() == target_text) 
#addresses the issue of exact matching by stripping the whitespaces and \ns near the target string"

if h2_element:
    #gets everything between <h2>Heading 2A</h2> and <h2>Heading 2B</h2>
    content = []

    #starts from the next element after the <h2>Heading 2A</h2>
    current_element = h2_element.find_next_sibling()

    while current_element and current_element.name != 'h2':
        content.append(str(current_element))
        current_element = current_element.find_next_sibling()

    # joins the content
    result = '\n'.join(content)

else:
    print(f"didnt find with {target_text} sed :()")

In [40]:
table = BeautifulSoup(result,"html.parser")
charge_capacity_all = table.find_all("td", class_ = "mw")#contains the other column("Original Charge")

#table.find_all("td", class_ = "dateTime") the dates are in week times and then rest in single days(not a full week)
if charge_capacity_all:
    
    charge_capacity_new = charge_capacity_all[::2]#with the other column removed

    charge_capacity_list = []
    
    for charge in charge_capacity_new:
        mwh = charge.prettify()
        charge_capacity_list.append(get_numbers_only(mwh,"mwh"))

else: 
    print("not found the mw class... please double check")

charge_capacity_list


['53024', '53070', '53135', '53135', '53135', '53135', '52937']

In [56]:
dates = table.find_all("td", class_ = "dateTime") 

if dates:

    date_list = []
    
    for date in dates:
        filtered_date = date.prettify()
        date_list.append(get_numbers_only(filtered_date,"date"))

else: 
    print("not found the mw class... please double check")

date_list

['2021-01-04',
 '2021-01-18',
 '2021-01-25',
 '2021-02-01',
 '2021-02-16',
 '2021-02-22',
 '2021-03-15',
 '2021-03-22',
 '2021-03-29',
 '2021-04-12',
 '2021-04-19',
 '2021-04-26',
 '2021-05-03',
 '2021-05-10',
 '2021-05-17',
 '2021-05-24',
 '2021-05-31',
 '2021-06-07',
 '2021-06-21',
 '2021-06-28',
 '2021-07-05',
 '2021-07-12',
 '2021-07-19',
 '2021-07-26',
 '2021-08-02',
 '2021-08-09',
 '2021-08-16',
 '2021-08-23',
 '2021-08-30',
 '2021-09-06',
 '2021-09-13',
 '2021-09-20',
 '2021-09-27',
 '2021-10-04',
 '2021-10-11',
 '2021-10-18',
 '2021-10-25',
 '2021-11-01',
 '2021-11-08',
 '2021-11-15',
 '2021-11-22',
 '2021-11-29',
 '2021-12-06',
 '2021-12-13',
 '2021-12-27',
 '2022-01-03',
 '2022-01-10',
 '2022-01-24',
 '2022-02-07',
 '2022-02-14',
 '2022-02-21',
 '2022-02-28',
 '2022-03-07',
 '2022-03-14',
 '2022-03-21',
 '2022-03-28',
 '2022-04-11',
 '2022-04-18',
 '2022-04-25',
 '2022-05-02',
 '2022-05-09',
 '2022-05-16',
 '2022-05-23',
 '2022-05-31',
 '2022-06-06',
 '2022-06-13',
 '2022-06-

In [58]:

interval_change_points = []

for i in range(1, len(date_list)):
    current_date = date_list[i]
    previous_date = date_list[i - 1]

    if current_date != previous_date:
        interval_change_points.append(i)

print("Interval change points at indices:", interval_change_points)


Interval change points at indices: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127]
